## Final Version - Performance Test (only model_5)

## PACKAGEs & MODULEs & FUNCTIONs

### Install Packages

In [1]:
# Install packages
#pip install plotly-express
#pip install ipython
#pip install zstandard
#pip install pyarrow
#pip install ipywidgets
#pip install ipykernel
#pip install --upgrade nbformat
#!pip install joblib
#!pip install --upgrade pip
#!pip install tensorflow
#%pip install joblib
#%pip install --upgrade pip
#%pip install tensorflow
#pip install tf-nightly

### Import Modules

In [2]:
# Import modules
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
sns.set(color_codes=True)
%matplotlib inline

import requests, zipfile, io
import os
import time

from utils_mod_v2 import *

import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.metrics import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.ensemble import IsolationForest

from sklearn.model_selection import train_test_split
import copy

from sklearn.preprocessing import MinMaxScaler

from numpy.random import seed
import tensorflow as tf
from tensorflow import keras

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from keras.callbacks import EarlyStopping

from scipy import stats
import random as random

import pickle

%load_ext autoreload
%autoreload 2

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
# Set random seeds for reproduction
seed=42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

### Define Functions

In [4]:
# Functions

## LOAD DATA

### Load data (full dataset)

In [5]:
# Read data
path_to_file = 'G:/My Drive/_My_Files_GoogleDrive/POS Data Science PUC/TCC/Datasets/joined_dataset_61.csv'

df_full = pd.read_csv(path_to_file)

df_full['Timestamp']= pd.to_datetime(df_full['Timestamp'])              # Set Timestamp as datetime
df_full.index = df_full['Timestamp']                                    # Set Timestamp as index (for future join on it)
df_full = df_full.drop(columns=['Timestamp'])                           # Drop Timestamp - not neede anymore
df_full = df_full.sort_index()                                          # Sort by index (Timestamp)
df_full = df_full[~df_full.index.duplicated(keep='last')]               # Remove any duplicated index
df_full

,uuid_835267,uuid_835268,uuid_835269,uuid_835270,uuid_835271,uuid_835272,uuid_835273,uuid_835274,uuid_835275,uuid_835276,uuid_835277,uuid_835278,uuid_835279,uuid_835280,uuid_835281,uuid_835282,uuid_835283,uuid_835284,uuid_835285,uuid_835286,uuid_835287,uuid_835288,uuid_835289,uuid_835290,uuid_835291,uuid_835292,uuid_835293,uuid_835294,uuid_835295,uuid_835296,uuid_835297,uuid_835298,uuid_835299,uuid_835300,uuid_835301,uuid_835302,uuid_835303,uuid_835304,uuid_835305,uuid_835306,uuid_835307,uuid_835308,uuid_835309,uuid_835310,uuid_835311,uuid_835312,uuid_835313,uuid_835314,uuid_835315,uuid_835316,uuid_835317,uuid_835318,uuid_835319,uuid_835320,uuid_835321,uuid_835322,uuid_835323,uuid_835324,uuid_835325,uuid_835326,uuid_835327
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-08 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-08 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-08 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-08 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-23 15:55:00,12111516.0,24972420.0,1229.0,1376466.0,66238.0,28746.0,4345.0,21970.0,1197.0,0.0,0.0,4.382856,2.803052,0.003001,523145049.0,4.120173e+09,3.118469e+09,305942843.0,105.488266,40.274260,7.270746,58.297297,761.529412,415.625342,46549.815730,65.448613,87.343212,0.055063,367215.760537,2865.665236,0.960280,0.024644,0.010570,0.565914,0.225076,0.023150,0.009739,0.005136,0.008696,0.953282,0.018002,0.054483,0.0,0.404096,0.0,0.000001,0.000624,0.001408,0.001408,4.008967,6376092.93,548413.02,6924505.95,1211.122207,145297.555736,8423.196467,5937.971091,2166.928180,237.278263,3000.392095,230.134889
2023-06-23 16:00:00,12058774.0,24294971.0,1118.0,1446675.0,66679.0,30066.0,4123.0,21003.0,1140.0,0.0,0.0,4.474455,2.958767,0.002795,467349956.0,3.994868e+09,2.993826e+09,348111516.0,105.086555,40.512629,7.415742,58.067348,0.000000,427.322533,46211.545974,65.301093,89.940497,0.055859,371573.369614,2841.497912,0.959814,0.024579,0.010462,0.565508,0.223830,0.023238,0.027315,0.005109,0.009818,0.955330,0.017030,0.054278,0.0,0.406323,0.0,0.000004,0.000619,0.001366,0.001366,4.014284,6280111.51,551579.98,6831691.49,1178.684554,145271.479074,8415.481224,6099.324533,2150.868796,237.491098,3004.949621,237.591039
2023-06-23 16:05:00,11819049.0,23416543.0,1265.0,1267406.0,61406.0,27416.0,3926.0,20140.0,1110.0,0.0,0.0,4.561457,2.671394,0.003084,451716797.0,3.799071e+09,2.947157e+09,305340170.0,104.966462,40.648256,7.435959,58.365061,0.000000,413.926468,46159.106529,65.744329,89.908646,0.055682,372556.357299,2864.582846,0.959429,0.024336,0.010369,0.566407,0.222213,0.022907,0.014178,0.004826,0.007266,0.952209,0.017783,0

## SET GLOBAL VARIABLES

### Define and Set Global Variables

In [6]:
# Set global variables

# ============================================================> Load Data
# Loading Data
LIST_OF_SERIES = ['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276',
                  'uuid_835277', 'uuid_835278', 'uuid_835279', 'uuid_835280', 'uuid_835281', 'uuid_835282', 'uuid_835283', 'uuid_835284', 'uuid_835285', 'uuid_835286',
                  'uuid_835287', 'uuid_835288', 'uuid_835289', 'uuid_835290', 'uuid_835291', 'uuid_835292', 'uuid_835293', 'uuid_835294', 'uuid_835295', 'uuid_835296',
                  'uuid_835297', 'uuid_835298', 'uuid_835299', 'uuid_835300', 'uuid_835301', 'uuid_835302', 'uuid_835303', 'uuid_835304', 'uuid_835305', 'uuid_835306',
                  'uuid_835307', 'uuid_835308', 'uuid_835309', 'uuid_835310', 'uuid_835311', 'uuid_835312', 'uuid_835313', 'uuid_835314', 'uuid_835315', 'uuid_835316',
                  'uuid_835317', 'uuid_835318', 'uuid_835319', 'uuid_835320', 'uuid_835321', 'uuid_835322', 'uuid_835323', 'uuid_835324', 'uuid_835325', 'uuid_835326',
                  'uuid_835327']

N_SAMPLES = 34560                                   # Define how many samples to work with (e.g.: 288=1Day, 8640=1Month, 34560=4Months)

# ============================================================> Train/Test Split
# Train, Validation and Test splitting
TRAIN_SPLIT = 0.9                                   # Define the percentage of dataset that will be use as training dataset

# ============================================================> Preprocessing prameters
# Replace extreme Outliers (to improve smoothing performance)
EXT_OUT_IQR_EPOCHS = 5                              # Define the number of epochs to run the extreme outliers replacement function
EXT_OUT_Z_EPOCHS = 5                                # Define the number of epochs to run the extreme outliers replacement function
REP_BY_DAYS_BACK = 1                                # When an outlier is found, it is replaced by the value that the same serie has "REP_BY_DAYS_BACK" days back in time.
Z_THRESHOLD = 3                                     # Define the threshold in z_score approach to remove extreme outliers
IQR_K = 1.5                                         # Define the Inter Quartile Range distance in the IQR approach to remove extreme outliers

# Smoothing process
SMOOTHING_EPOCHS = 15                               # Define the number of epochs to run the smoothing function
ROLLING_WINDOW_SIZE = 12                            # Define the size of the rolling window
ROLLING_WINDOW_STD_THRESHOLD = 2                    # Define the std threshold in rolling windows approach to smooth the series
ROLLING_WINDOW_CENTER=True                          # Centers the rolling window

# Reshaping (2D to 3D - windows)
WIN_SIZE = 12                                       # Define the size of the window to train on each sample at once - lookback or lookforward.
WIN_DIR = 'backward'                                # Direction of the window (backward = WIN_SIZE samples to the past | forward = WIN_SIZE samples to the future)

# ============================================================> Model prameters
# Trainning parameters
BATCH_SIZE = int(df_full.shape[0]*TRAIN_SPLIT/100)  # Define the size of the batch
N_EPOCHS = 100                                      # Define the number of epochs to run the algorithm
VAL_SPLIT = 0.05                                    # Define the percentage of the training set that will be used as validation dataset
LOSS = 'mse'                                        # Define the loss function to be used (mae or mse)
OPTIMIZER = 'nadam'                                 # Define the optimizaer to be used (adam, adadelta, nadam, RMSprop)
ANOMALY_LAYER ='no'                                 # Whether to use the anomaly layer ==> Future use "need y_true/y_pred" to use the anomaly vector to set custom loss function.
EARLY_STOPPING = True                               # Whether to use EarlyStopping or not.
PATIENCE = 5                                        # EarlyStopping patience.
MONITOR = 'loss'                                    # EarlyStopping monitoring parameter.
MIN_DELTA = 0.0001                                  # EarlyStopping min_delta.

# ============================================================> Anomaly Detection Parameters
# Anomaly detection parameters
DIST = 'sqr'                                        # Define the distance function to be used (abs or sqr)
ANOMALY_COUNT_THRESHOLD = 2                         # Define the number of simoultaneous anomalous series
SENSITIVITY_PER_OF_MAX = 0.99                       # Define the percentage of the max(avg(loss)) of training as threshold
SENSITIVITY_PER_OF_SUM = 0.99                       # Define the percentage of the sum(sort(avg(loss))) of training as threshold
THRESHOLD_TO_USE = 'Individual'                     # Whether to use Individual or Average thresholds during detection

## ANALYZE DATA

### Select data to be used

In [7]:
# Select a subset of series
df_full = df_full[LIST_OF_SERIES]
print('Number of Timeseries: ' + str(len(LIST_OF_SERIES)))

Number of Timeseries: 61


In [8]:
# Select keep only last x samples (4 months)
df_full = df_full[-N_SAMPLES:]
print('Lenght of Timeseries: ' + str(N_SAMPLES))

Lenght of Timeseries: 34560


## RUN PERFORMANCE TEST

In [9]:
# Create a list of series (increasing series)
SERIES = []
INIT_NUM_OF_SERIES = 9
STEP_NUM_OF_SERIES = 5

for i in range(INIT_NUM_OF_SERIES,len(LIST_OF_SERIES),STEP_NUM_OF_SERIES):
    increasing_series = LIST_OF_SERIES[:i+1]
    SERIES.append(increasing_series)

In [10]:
# Inspect list of series
for series in SERIES:
    print(series)

['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276']
['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276', 'uuid_835277', 'uuid_835278', 'uuid_835279', 'uuid_835280', 'uuid_835281']
['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276', 'uuid_835277', 'uuid_835278', 'uuid_835279', 'uuid_835280', 'uuid_835281', 'uuid_835282', 'uuid_835283', 'uuid_835284', 'uuid_835285', 'uuid_835286']
['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276', 'uuid_835277', 'uuid_835278', 'uuid_835279', 'uuid_835280', 'uuid_835281', 'uuid_835282', 'uuid_835283', 'uuid_835284', 'uuid_835285', 'uuid_835286', 'uuid_835287', 'uuid_

In [11]:
# Run performance experiments
PRE_PRO_TIME = []
TRAINING_TIME = []
i = 0

print('\n\n')
print('==========================================================================================')
print('==========================================================================================')
print('LSTM AE Models Performane Experiments')
print('==========================================================================================')
print('==========================================================================================\n\n')

for s in SERIES:
    i+=1
    print('Performance experiment: ' + str(i) + ' out of ' + str(len(SERIES)))
    print('Number of Timeseries the algorithm will be traing with: ' + str(len(s)) + '\n')
    print('List of Timeseries: ' + str(s))

    # Read data
    print('Loading dataset...')
    print('==========================================================================================')
    path_to_file = 'G:/My Drive/_My_Files_GoogleDrive/POS Data Science PUC/TCC/Datasets/joined_dataset_61.csv'
    df_full = pd.read_csv(path_to_file)
    df_full['Timestamp']= pd.to_datetime(df_full['Timestamp'])              # Set Timestamp as datetime
    df_full.index = df_full['Timestamp']                                    # Set Timestamp as index (for future join on it)
    df_full = df_full.drop(columns=['Timestamp'])                           # Drop Timestamp - not neede anymore
    df_full = df_full.sort_index()                                          # Sort by index (Timestamp)
    df_full = df_full[~df_full.index.duplicated(keep='last')]               # Remove any duplicated index
    df_full = df_full[-N_SAMPLES:]
    print('... Done!\n')

    # Define DataSet
    df_full = df_full[s]

    # Training & Test split
    print('Training & Test splitset...')
    print('==========================================================================================')
    start_pre_pro = time.time()
    train = df_full[:int(np.floor(TRAIN_SPLIT * df_full.shape[0]))]
    test = df_full[int(np.floor(TRAIN_SPLIT * df_full.shape[0])):int(np.floor(df_full.shape[0]))]
    print('... Done!\n')

    # Remove extreme outliers - IQR
    print('Remove extreme outliers - IQR...')
    print('==========================================================================================')
    df_cleaned_iqr = replace_outliers_iqr(data=train,
                                            iqr_k=IQR_K,
                                            rep_by_days_back=REP_BY_DAYS_BACK,
                                            epochs=EXT_OUT_IQR_EPOCHS)
    print('... Done!\n')

    # Remove extreme outliers - Z-Score
    print('Remove extreme outliers - Z-Score...')
    print('==========================================================================================')
    df_cleaned_z_score = replace_outliers_z_score(data=df_cleaned_iqr,
                                                    z_threshold=Z_THRESHOLD,
                                                    rep_by_days_back=REP_BY_DAYS_BACK,
                                                    epochs=EXT_OUT_Z_EPOCHS)
    print('... Done!\n')

    # Remove outliers from training dataset using rolling windows approach - new version of the function (vectorized smoothing)
    print('Smoothing process...')
    print('==========================================================================================')
    train_smoothed = remove_outliers_rolling_windows_vec(df=df_cleaned_z_score,
                                                            rolling_window_size=ROLLING_WINDOW_SIZE,
                                                            epochs=SMOOTHING_EPOCHS,
                                                            rolling_window_std_threshold=ROLLING_WINDOW_STD_THRESHOLD,
                                                            center=ROLLING_WINDOW_CENTER)
    print('... Done!\n')

    # Normalize data - Apply scaler MinMaxScaler()
    print('Normalize data...')
    print('==========================================================================================')
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(train_smoothed)
    X_test = scaler.transform(test)
    print('... Done!\n')

    # Reshape inputs for LSTM [samples, timesteps, features]
    print('Reshape inputs (create 3D structure)...')
    print('==========================================================================================')
    X_train_3d = transform_to_windows(X=X_train, win_size=WIN_SIZE, win_dir=WIN_DIR)
    X_test_3d = transform_to_windows(X=X_test, win_size=WIN_SIZE, win_dir=WIN_DIR)
    print('... Done!\n')
    end_pre_pro = time.time()

    # Create the autoencoder model_v5 (instantiation) | WINDOWS approach!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    LSTM_AE_model_v5, early_stopping_model_v5 = lstm_ae_model_v5(X_train_3d,
                                                                anomaly_layer=ANOMALY_LAYER,
                                                                early_stopping=EARLY_STOPPING,
                                                                monitor=MONITOR,
                                                                patience=PATIENCE,
                                                                min_delta=MIN_DELTA)
    LSTM_AE_model_v5.compile(optimizer=OPTIMIZER, loss=LOSS)

    # Train the model LSTM_AE_model_v5 | WINDOWS approach!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    print('Train the model...')
    print('==========================================================================================')
    start_training = time.time()
    history_LSTM_AE_model_v5 = LSTM_AE_model_v5.fit(X_train_3d,
                                                    X_train_3d,
                                                    epochs=N_EPOCHS,
                                                    batch_size=BATCH_SIZE, 
                                                    validation_split=VAL_SPLIT,
                                                    shuffle=False,
                                                    callbacks=[early_stopping_model_v5]).history
    end_training = time.time()
    print('... Done!\n')

    # Save models
    LSTM_AE_model_v5.save('LSTM_AE_model_v11_2_'+str(i)+'.h5')

    # Save Training History
    with open('history_LSTM_AE_model_v5_11_2_'+str(i)+'.pkl', 'wb') as file:
        pickle.dump(history_LSTM_AE_model_v5, file)

    print('Preprocessing elapsed time: ' + str(end_pre_pro - start_pre_pro))
    print('Model Training elapsed time: ' + str(end_training - start_training))
    
    PRE_PRO_TIME.append(str(end_pre_pro - start_pre_pro))
    TRAINING_TIME.append(str(end_training - start_training))
    print('\n\n')

print('\n\n')
print('==========================================================================================')
print('==========================================================================================')
print('Summary')
print('==========================================================================================')
print('==========================================================================================')
print('\n')

for i in range(len(PRE_PRO_TIME)):
    print('Number of Timeseries :           ' + str(len(SERIES[i])))
    print('Preprocessing elapsed time:      ' + str(PRE_PRO_TIME[i]))
    print('Model Training elapsed time:     ' + str(TRAINING_TIME[i]))
    print('\n')




LSTM AE Models Performane Experiments


Performance experiment: 1 out of 11
Number of Timeseries the algorithm will be traing with: 10

List of Timeseries: ['uuid_835267', 'uuid_835268', 'uuid_835269', 'uuid_835270', 'uuid_835271', 'uuid_835272', 'uuid_835273', 'uuid_835274', 'uuid_835275', 'uuid_835276']
Loading dataset...
... Done!

Training & Test splitset...
... Done!

Remove extreme outliers - IQR...
IQR (Interquartile Range) => Epoch #1: Replacing 3 in uuid_835267
IQR (Interquartile Range) => Epoch #1: Replacing 0 in uuid_835268
IQR (Interquartile Range) => Epoch #1: Replacing 181 in uuid_835269
IQR (Interquartile Range) => Epoch #1: Replacing 4 in uuid_835270
IQR (Interquartile Range) => Epoch #1: Replacing 34 in uuid_835271
IQR (Interquartile Range) => Epoch #1: Replacing 219 in uuid_835272
IQR (Interquartile Range) => Epoch #1: Replacing 557 in uuid_835273
IQR (Interquartile Range) => Epoch #1: Replacing 1 in uuid_835274
IQR (Interquartile Range) => Epoch #1: Replacing 32 i